In [22]:
from random import choices, choice, shuffle
from typing import List
from datetime import date

from arr import Contract, ContractHeader, ContractLine

import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta as rd

In [23]:
MIN_DATE = date(2020, 1, 1)
MAX_DATE = date(2026, 12, 31)
START_DATES = pd.date_range(MIN_DATE, MAX_DATE)
RENEWAL_CHANCE = .8
EXPANSION_CHANCE = .1
DOWNGRADE_CHANCE = .1

CUSTOMERS = pd.read_excel('data/saas_corp.xlsx','customer')
CUSTOMERS.set_index('key', inplace=True)

PRODUCTS = pd.read_excel('data/saas_corp.xlsx','product')
PRODUCTS.set_index('key', inplace=True)
PRODUCT_WEIGHTS = np.arange(0,1,1/len(PRODUCTS))[::-1]
PRODUCT_COUNT = len(PRODUCTS['product_name'].unique())

CONTRACTS = pd.read_excel('data/saas_corp.xlsx', 'contract')

In [24]:
def random_contract_range(
    contract_lengths: List[int] = [3, 6, 12, 24, 36],
    contract_weights: List[float] = [0.1, 0.05, 0.6, 0.2, 0.05],
) -> int:
    """Get a random contract length interval.

    Wrapper for:
    https://docs.python.org/3/library/random.html#random.choices

    Args:
        contract_lengths (List[int], optional): Possible contract lengths, in months.
            Defaults to [3, 6, 12, 24, 36].
        contract_weights (List[float], optional): Weights of the `contract_lengths`, arg.
            Defaults to [0.1, 0.05, 0.6, 0.2, 0.05].

    Returns:
        int: Length of contract in months.
    """
    return choices(contract_lengths, contract_weights)[0]

In [25]:
def initial_sale(customer: str) -> Contract:
    """Get customer start, end dates. Get customer items for first sale"""

    start_date = choice(START_DATES).date()
    contract_range = random_contract_range()
    end_date = start_date + rd(months=contract_range, days=-1)

    num_of_items = choices(range(1, PRODUCT_COUNT + 1), PRODUCT_WEIGHTS)[0]
    products = PRODUCTS.loc[np.random.choice(PRODUCTS.index, num_of_items, False)]

    contract = Contract(
        id=1,
        header=ContractHeader(products["amount"].sum(), start_date, end_date),
        lines=[
            ContractLine(
                row[1]["amount"],
                start_date,
                end_date,
                row[1]["product_name"],
                row[1]["renewable"],
            )
            for row in products.iterrows()
        ],
        customer=customer,
    )

    return contract

In [28]:
all_contracts = []

for customer in CUSTOMERS['customer'].unique():
    if customer in CONTRACTS["customer"].unique():
        print(f"{customer} already in contract list")
    else:
        all_contracts.append(initial_sale(customer).to_df())

first_contract = pd.concat(all_contracts)
first_contract

Abatz already in contract list


,id,customer,header.amount,header.start_date,header.end_date,header.booking_date,line.amount,line.start_date,line.end_date,line.product,line.renewable
0,1,Topicshots,43500,2023-05-27,2024-05-26,2023-05-27,8500,2023-05-27,2024-05-26,ITS,True
1,1,Topicshots,43500,2023-05-27,2024-05-26,2023-05-27,25000,2023-05-27,2024-05-26,ERP,True
2,1,Topicshots,43500,2023-05-27,2024-05-26,2023-05-27,10000,2023-05-27,2024-05-26,CRM,True
0,1,Twinte,113500,2021-11-04,2022-11-03,2021-11-04,5000,2021-11-04,2022-11-03,CMS,True
1,1,Twinte,113500,2021-11-04,2022-11-03,2021-11-04,7500,2021-11-04,2022-11-03,DevOps,True
...,...,...,...,...,...,...,...,...,...,...,...
4,1,Lazz,62500,2020-06-13,2022-06-12,2020-06-13,6500,2020-06-13,2022-06-12,SCM,True
0,1,Nlounge,45000,2020-12-14,2021-12-13,2020-12-14,6000,2020-12-14,2021-12-13,IAM,True
1,1,Nlounge,45000,2020-12-14,2021-12-13,2020-12-14,7500,2020-12-14,2021-12-13,DevOps,True
2,1,Nlounge,45000,2020-12-14,2021-12-13,2020-12-14,6500,2020-12-14,2021-12-13,SCM,True


In [ ]:
first_contract[['customer','header.end_date','line.product']].groupby(['customer','header.end_date']).count()
# use the # of products to determine if more products can be added.

,,line.product
customer,header.end_date,
Avaveo,2021-08-03,10
Babbleset,2027-07-23,8
Blogtag,2022-09-02,2
Bluezoom,2023-03-01,2
Brainverse,2026-05-09,7
...,...,...
Zazio,2024-04-09,2
Zoomlounge,2026-10-02,12
Zoomzone,2022-07-27,6
